In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# pd.set_option("display.max_rows", None)

In [2]:
end = datetime(2024, 3, 30)
options = pd.read_csv(r"data\cleaned_options_data.csv")


# Step 1: Convert the 'ts_recv' column to datetime (this is already optimized)
options["datetime"] = pd.to_datetime(options["ts_recv"], format="%Y-%m-%dT%H:%M:%S.%fZ")


# Step 2: Vectorized parsing of the 'symbol' column
# Split by spaces and extract the relevant parts directly
symbol_parts = options["symbol"].str.split(" ", expand=True)[1]


# Extract expiration date (first 6 characters)
options["exp_date"] = pd.to_datetime(symbol_parts.str[:6], format="%y%m%d")
options = options[options["exp_date"] <= end]


# Extract action ('C' or 'P', at index 6)
options["action"] = symbol_parts.str[6]


# Extract strike price (from index 7 onward, divided by 1000)
options["strike_price"] = symbol_parts.str[7:].astype(float) / 1000


# Step 3: Rename the other columns as needed
options = options.rename(
    columns={
        "bid_px_00": "bidp",
        "ask_px_00": "askp",
        "bid_sz_00": "bid_sz",
        "ask_sz_00": "ask_sz",
    }
)

options["datetime"] = (
    pd.to_datetime(options["datetime"])
    .dt.tz_localize("UTC")
    .dt.tz_convert("US/Eastern")
    .dt.tz_localize(None)
)
options["date"] = options["datetime"].dt.date


# Calculate 'till_exp' (time to expiration in years)
options["till_exp"] = (options["exp_date"] - options["datetime"]).dt.days / 365.0


# Calculate 'fair_value' as the average of bid and ask price
options["fair_value"] = (options["bidp"] + options["askp"]) / 2.0

In [3]:
underlying = pd.read_csv(r"data\spx_minute_level_data_jan_mar_2024.csv")
underlying = underlying.loc[underlying["price"] > 0]
underlying["date"] = pd.to_datetime(underlying["date"], format="%Y%m%d")
underlying_min = underlying.copy()

underlying = (
    underlying.groupby("date")
    .agg(
        Open=("price", "first"),
        High=("price", "max"),
        Low=("price", "min"),
        Close=("price", "last"),
    )
    .reset_index()
)

In [4]:
underlying_min["date"] = pd.to_datetime(underlying_min["date"]).dt.date
time_part = pd.to_timedelta(underlying_min["ms_of_day"], unit="ms")
underlying_min["hour"] = time_part.dt.components.hours
underlying_min["minute"] = time_part.dt.components.minutes

In [5]:
indices = underlying["date"].searchsorted(options["datetime"]) - 1
indices = indices.clip(0, len(underlying) - 1)
options["underlying"] = underlying["Close"].iloc[indices].values

In [6]:
# For calls, I want difference to be as high as possible
# For puts, I want difference to be as negative as possible
options["deviate"] = options["underlying"] - options["strike_price"]

# Make options["deviate"] negative for calls
options.loc[options["action"] == "P", "deviate"] *= -1

In [7]:
options.head(3)

,ts_recv,instrument_id,bidp,askp,bid_sz,ask_sz,symbol,datetime,exp_date,action,strike_price,date,till_exp,fair_value,underlying,deviate
0,2024-01-02T14:30:02.402838204Z,587228771,27.0,27.70,174,155,SPX 240119P04700000,2024-01-02 09:30:02.402838204,2024-01-19,P,4700.0,2024-01-02,0.043836,27.350,4743.14,-43.14
1,2024-01-02T14:30:02.402844171Z,587211744,1.6,1.75,755,1981,SPX 240119P04300000,2024-01-02 09:30:02.402844171,2024-01-19,P,4300.0,2024-01-02,0.043836,1.675,4743.14,-443.14
2,2024-01-02T14:30:02.402848382Z,587211728,4.1,4.40,1566,1125,SPX 240119P04500000,2024-01-02 09:30:02.402848382,2024-01-19,P,4500.0,2024-01-02,0.043836,4.250,4743.14,-243.14


In [9]:
# Create a loop over unique expiration dates
unique_exp_dates = options["exp_date"].unique()

capital = 100_000_000
threshold = -150
orders = pd.DataFrame(columns=["datetime", "option_symbol", "action", "order_size"])


for exp_date in unique_exp_dates:
    active_ord = pd.DataFrame(
        columns=[
            "strike_price",
            "order_size",
            "datetime",
            "exp_date",
            "action",
        ]
    )
    day_before_exp = (exp_date - pd.Timedelta(days=1)).date()
    filtered_options = options[
        (options["date"] == day_before_exp) & (options["exp_date"] == exp_date)
    ]
    opp_options = filtered_options.sort_values(by="deviate", ascending=True)

    # Loop through opp options
    for index, option in opp_options.iterrows():
        # General stuff
        if option["deviate"] > threshold:
            break
        if option["bid_sz"] == 0:
            continue
        margin = (
            0.1 * option["strike_price"]
            if option["action"] == "C"
            else 0.1 * option["underlying"]
        )
        order_size = min(option["bid_sz"], (capital - margin) / (option["bidp"] * 100))
        if order_size <= 0:
            break
        earnings = order_size * 100 * option["bidp"]
        if earnings < 1:
            continue

        if capital >= margin + earnings:
            capital += order_size * option["bidp"] * 100

        new_row = pd.DataFrame(
            [
                {
                    "datetime": option["ts_recv"],  # ts_recv for the Call option
                    "option_symbol": option["symbol"],  # Use the Call option symbol
                    "action": "S",  # Action for the Call option
                    "order_size": order_size,  # Minimum ask size
                }
            ]
        )
        new_row2 = pd.DataFrame(
            [
                {
                    "strike_price": option["strike_price"],
                    "order_size": order_size,
                    "datetime": option["datetime"],
                    "exp_date": option["exp_date"],
                    "action": option["action"],
                }
            ]
        )
        orders = pd.concat(
            [orders, new_row],
            ignore_index=True,
        )
        active_ord = pd.concat(
            [active_ord, new_row2],
            ignore_index=True,
        )
        # print(
        #     option["sort_by"],
        #     option["strike_price"],
        #     option["underlying"],
        #     option["bidp"],
        # )

    # Loop through active orders on expiration date and figure out prices
    active_ord["date"] = pd.to_datetime(active_ord["exp_date"]).dt.date

    # Create hour and minute columns but force them between 9:31 and 16:00
    active_ord["datetime"] = pd.to_datetime(active_ord["datetime"])
    active_ord["hour"] = active_ord["datetime"].dt.hour
    active_ord["minute"] = active_ord["datetime"].dt.minute
    active_ord.loc[(active_ord["hour"] < 9), "hour"] = 9
    active_ord.loc[
        (active_ord["hour"] == 9) & (active_ord["minute"] < 31), ["hour", "minute"]
    ] = [9, 31]
    active_ord.loc[(active_ord["hour"] > 16), ["hour", "minute"]] = [16, 0]

    # Merge with underlying_min to get the underlying price
    active_ord = active_ord.merge(
        underlying_min, how="left", on=["date", "hour", "minute"]
    )
    # print(active_ord.head(5))
    # break
    for index, option in active_ord.iterrows():
        diff = option["price"] - option["strike_price"]
        if option["action"] == "C":
            if diff > 0:
                capital -= option["order_size"] * 100 * diff
        else:
            if diff < 0:
                capital -= option["order_size"] * 100 * -diff

# Backtesting Code


In [10]:
from datetime import datetime
from backtester import Backtester
from feed_strategy import Strategy

start = datetime(2024, 1, 1)
end = datetime(2024, 3, 30)
s = Strategy(orders)
b = Backtester(start, end, s)

In [ ]:
b.calculate_pnl()

In [ ]:
b.plot_pnl()

In [ ]:
b.compute_overall_score()